# Регрессионная модель изменения цен на дома в Бостоне

# Выполнила Матюшина Алина БСТ1802


# Цель
Реализовать предсказание медианной цены на дома в пригороде Бостона в середине 1970-х по таким данным, как уровень преступности, ставка местного имущественного налога и т.д.
Данный набор содержит относительно немного образцов данных: всего 506, разбитых на 404 обучающих и 102 контрольных образца. И каждый признак во входных данных(например, уровень преступности) имеет свой масштаб. Например, некоторые признаки
являются пропорциями и имеют значения между 0 и 1, другие — между 1 и 12 и т. д.
Не путайте регрессию с алгоритмом логистической регрессии. Как ни странно, логистическая регрессия не является регрессионным алгоритмом — это алгоритм классификации.

# Задачи

* Ознакомиться с задачей регрессии
* Изучить отличие задачи регрессии от задачи классификации
* Создать модель
* Настроить параметры обучения
* Обучить и оценить модели
* Ознакомиться с перекрестной проверкой

# Выполнение работы


### Подключение модулей

In [1]:
import numpy as np
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.datasets import boston_housing

import matplotlib.pyplot as plt

### Загрузка данных

Загрузим данные по поставленным целям и выведим их

In [2]:
(train_data, train_targets), (test_data, test_targets) = boston_housing.load_data()
print(train_data.shape)
print(test_data.shape)
print(test_targets)

(404, 13)
(102, 13)
[ 7.2 18.8 19.  27.  22.2 24.5 31.2 22.9 20.5 23.2 18.6 14.5 17.8 50.
 20.8 24.3 24.2 19.8 19.1 22.7 12.  10.2 20.  18.5 20.9 23.  27.5 30.1
  9.5 22.  21.2 14.1 33.1 23.4 20.1  7.4 15.4 23.8 20.1 24.5 33.  28.4
 14.1 46.7 32.5 29.6 28.4 19.8 20.2 25.  35.4 20.3  9.7 14.5 34.9 26.6
  7.2 50.  32.4 21.6 29.8 13.1 27.5 21.2 23.1 21.9 13.  23.2  8.1  5.6
 21.7 29.6 19.6  7.  26.4 18.9 20.9 28.1 35.4 10.2 24.3 43.1 17.6 15.4
 16.2 27.1 21.4 21.5 22.4 25.  16.6 18.6 22.  42.8 35.1 21.5 36.  21.9
 24.1 50.  26.7 25. ]


Проведем нормализацию, для облегчения обучения нейроной сети. 
Для каждого признака во входных данных (столбца в матрице входных данных) из каждого значения вычитается среднее по этому признаку, и разность делится на стандартное отклонение, в результате признак центрируется по нулевому значению и имеет стандартное отклонение, равное единице. Такую нормализацию легко выполнить с помощью Numpy.

In [3]:
# Находим среднее значение(сз) для каждого столбца 
mean = train_data.mean(axis=0)
# Вычисляем отклонение от сз для каждого столбца
train_data -= mean
# Вычисляем стандартное отклонение для каждого столбца
std = train_data.std(axis=0)
# Централизация данных
train_data /= std 

# Централизация и нормирование тестовых данных
test_data -= mean #
test_data /= std #

Определим функцию build_model() в которой:
* Определим последовательность модели нейронной сети (от меньшего к большему)
* Добавление слоя модели с использованием функции активации ReLu (возвращает значение х, если х положительно, иначе 0)
* Добавления слоя с предсказание "цены" в любом деапозоне
* Добавление параметра обучения:
    1. Оптимизация - Сохранение скользящее среднее значение квадрата градиентов и резделение градиента на корень из этого среднего значения
    2. Потери - Средняя квадратическая ошибка
    3. Метрика для мониторинга на этапах обучения и тестирования - средняя абсолютная ошибка (абсолютное значение разности между целевым и вычисленным значениями)

In [4]:
def build_model():
    model = Sequential()
    # input_shape - указание на то что, на вход идет 13 параметров 
    model.add(Dense(64, activation='relu', input_shape=(train_data.shape[1],)))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(1))
    model.compile(optimizer='rmsprop', loss='mse', metrics=['mae'])
    return model

Хорошей практикой в таких ситуациях является применение перекрестной проверки по K
блокам (K-fold cross-validation). Суть ее заключается в разделении доступных данных на K
блоков (обычно K = 4 или 5), создании K идентичных моделей и обучении каждой на K—1
блоках с оценкой по оставшимся блокам. По полученным K оценкам вычисляется среднее
значение, которое принимается как оценка модели. В коде такая проверка реализуется
достаточно просто.

In [6]:
def elsTask(k, num_epochs):
    num_val_samples = len(train_data) // k # строк в каждом блоке
    all_scores = [] # Список результатов работ 

    acc = []
    loss = []

    for i in range(k):
        print('processing fold #', i)
        #Выбор данных для тестовой нейроной сети
        val_data = train_data[i * num_val_samples: (i + 1) * num_val_samples]
        val_targets = train_targets[i * num_val_samples: (i + 1) * num_val_samples]

        #Выбор данных для обучения нейроной сеити
        partial_train_data = np.concatenate([train_data[:i * num_val_samples], train_data[(i + 1) * num_val_samples:]],axis=0)
        partial_train_targets = np.concatenate([train_targets[:i * num_val_samples], train_targets[(i + 1) * num_val_samples:]], axis=0)

        # Создание модели нейросети
        model = build_model()

        # Адаптации модели под обучающие данные
        d = model.fit(partial_train_data, partial_train_targets, epochs=num_epochs, batch_size=1, verbose=0)

        #Оценка модели
        #Вычисляет значение потерь и значения всех метрик, которые мы выбрали при составлении модели.
        val_mse, val_mae = model.evaluate(val_data, val_targets, verbose=1)

        #Добавление результатов работы в список 
        all_scores.append(val_mae)


        acc.append(d.history["mae"])
        loss.append(d.history["loss"])

        
    print (np.mean(all_scores))
    
    #Построим графики Абсолютное отклонение по всем моделям в ходе обучения
    x = [x for x in range(num_epochs)]
    
    if k == 1: 
        plt.plot(x, acc[0], 'red')
    elif k == 2: 
        plt.plot(x, acc[0], 'red', x, acc[1], 'green')
        plt.legend(['red - first model','green - second model','blue - third model'])
    elif k == 3: 
        plt.plot(x, acc[0], 'red', x, acc[1], 'green', x, acc[2], 'blue')
        plt.legend(['red - first model','green - second model','blue - third model'])
    elif k == 4: 
        plt.plot(x, acc[0], 'red', x, acc[1], 'green', x, acc[2], 'blue', x, acc[3], 'black')
        plt.legend(['red - first model','green - second model','blue - third model','black - fourth model'])
    else: 
        plt.plot(x, acc[0], 'red', x, acc[1], 'green', x, acc[2], 'blue', x, acc[3], 'black', x, acc[3], 'pink')
        plt.legend(['red - first model','green - second model','blue - third model','black - fourth model', 'pink - fifth model'])
    
    plt.ylabel("Абсолютная ошибка")
    plt.xlabel("Эпоха")
    plt.title("Абсолютное отклонение по всем моделям в ходе обучения")
    plt.show()

    #Усредненное абсолютное отклонение
    y = []
    for i in range(num_epochs):
        y.append(np.mean([acc[j][i] for j in range(k)]))

    plt.plot(x, y, 'red')
    plt.ylabel("Абсолютная ошибка (среднее)")
    plt.xlabel("Эпоха")
    plt.title("Усредненное абсолютное отклонение")
    plt.show()

    #Среднеквадратичная ошибка по всем моделям в ходе обучения
    if k == 1: 
        plt.plot(x, loss[0], 'red')
    elif k == 2: 
        plt.plot(x, loss[0], 'red', x, loss[1], 'green')
        plt.legend(['red - first model','green - second model','blue - third model'])
    elif k == 3: 
        plt.plot(x, loss[0], 'red', x, loss[1], 'green', x, loss[2], 'blue')
        plt.legend(['red - first model','green - second model','blue - third model'])
    elif k == 4: 
        plt.plot(x, loss[0], 'red', x, loss[1], 'green', x, loss[2], 'blue', x, loss[3], 'black')
        plt.legend(['red - first model','green - second model','blue - third model','black - fourth model'])
    else: 
        plt.plot(x, loss[0], 'red', x, loss[1], 'green', x, loss[2], 'blue', x, loss[3], 'black', x, loss[3], 'pink')
        plt.legend(['red - first model','green - second model','blue - third model','black - fourth model', 'pink - fifth model'])
    plt.ylabel("Среднеквадратичная ошибка (отклонение)")
    plt.xlabel("Эпоха")
    plt.title("Среднеквадратичная ошибка по всем моделям в ходе обучения")
    plt.show()

    #Усредненная среднеквадратичная ошибка
    y = []
    for i in range(num_epochs):
        y.append(np.mean([loss[j][i] for j in range(k)]))

    plt.plot(x, y, 'red', )
    plt.ylabel("Среднеквадратичная ошибка (отклонение) (среднее)")
    plt.xlabel("Эпоха")
    plt.title("Средненная среднеквадратичная ошибка")
    plt.show()
    

IndentationError: unindent does not match any outer indentation level (<tokenize>, line 71)

In [ ]:
elsTask(1,100)

Проведем повторное моделирование, но с увеличением кол-во эпох

In [ ]:
elsTask(4,150)

Увеличем количество блоков

In [ ]:
elsTask(6,100)

# Выводы
Самой точно моделью оказалась та, которая имеет больше блоков, при одинаковом количестве эпох. При увлечение эпох в 1.5 раза, изменилось абсолютное отклонение в лучшую сторону, но не кретически. При увеличение количество блоков на один, так же привело к снижению абсолютного отклонения. Таким образом, нужно стремится к "балансу" между колличеством эпох и колличеством блоков, для того чтобы прийти к оптимальным значениям и неперегружать систему.
